# **빅데이터 개론 Lab 11 - 교차분석, 데이터베이스 연동**

참고자료 : https://www.notion.so/TA-2689a38b5289413a82671d3956fea103

- - -




### **<교차분석>**

* 두 범주형 변수 간에 연관관계(association)가 있는지의 여부를 판단하고자 하는 경우 교차표를 작성하여 변수들 간의 관계를 분석한다.

 - 이를 교차분석 혹은 χ2(chi-square) 검정(test)이라고 한다. 교차분석은 두 변수의 빈도표를 교차 시킨다는 의미로 해석되며, 교차분석에 사용되는 검정 통계량이 ``χ2 -분포를 따르므로 χ2 - 검정``으로 부른다.

 - 교차표(cross tabulation), 분할표(contingency table): 각 범주형 변수에 대한 빈도표를 행과 열로 결합시켜 놓은 형태이다. 일반적으로 행에는 설명(독립)변수에 해당 하는 변수를 할당하고, 열에는 반응(종속)변수를 할당한다. 


In [ ]:
install.packages(c("tidyverse", "data.table", "gmodels"))
library(tidyverse)
library(data.table)
library(gmodels)
options(repr.plot.width=13, repr.plot.height=13)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gtools’, ‘gdata’


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




#### **A. 예제 1**

---


1. **HR(Human Resource, 인사관리)_연습데이터([링크](https://www.dropbox.com/sh/xx1w2syi768kfu0/AACZgxgo1fcxyDMgv9U-iTz8a?dl=0))**

* 변수 설명

```
satisfaction_level : 직무 만족도
last_evaluation : 마지막 평가점수
number_project : 진행 프로젝트 수
average_monthly_hours : 월평균 근무시간
time_spend_company : 근속년수
work_accident : 사건사고 여부(0: 없음, 1: 있음, 범주형)
left : 이직 여부(0: 잔류, 1: 이직, 범주형)
promotion_last_5years: 최근 5년간 승진여부(0: 승진 x, 1: 승진, 범주형)
sales : 부서(범주형)
salary : 임금 수준(범주형)
```


In [ ]:
hr_DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/HR_comma_sep.csv", sep = ",") %>% as_tibble()

In [ ]:
hr_DF %>% str()

tibble [14,999 × 10] (S3: tbl_df/tbl/data.frame)
 $ satisfaction_level   : num [1:14999] 0.38 0.8 0.11 0.72 0.37 0.41 0.1 0.92 0.89 0.42 ...
 $ last_evaluation      : num [1:14999] 0.53 0.86 0.88 0.87 0.52 0.5 0.77 0.85 1 0.53 ...
 $ number_project       : int [1:14999] 2 5 7 5 2 2 6 5 5 2 ...
 $ average_montly_hours : int [1:14999] 157 262 272 223 159 153 247 259 224 142 ...
 $ time_spend_company   : int [1:14999] 3 6 4 5 3 3 4 5 5 3 ...
 $ Work_accident        : int [1:14999] 0 0 0 0 0 0 0 0 0 0 ...
 $ left                 : int [1:14999] 1 1 1 1 1 1 1 1 1 1 ...
 $ promotion_last_5years: int [1:14999] 0 0 0 0 0 0 0 0 0 0 ...
 $ sales                : chr [1:14999] "sales" "sales" "sales" "sales" ...
 $ salary               : chr [1:14999] "low" "medium" "medium" "low" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [ ]:
hr_DF %>% summary()

 satisfaction_level last_evaluation  number_project  average_montly_hours
 Min.   :0.0900     Min.   :0.3600   Min.   :2.000   Min.   : 96.0       
 1st Qu.:0.4400     1st Qu.:0.5600   1st Qu.:3.000   1st Qu.:156.0       
 Median :0.6400     Median :0.7200   Median :4.000   Median :200.0       
 Mean   :0.6128     Mean   :0.7161   Mean   :3.803   Mean   :201.1       
 3rd Qu.:0.8200     3rd Qu.:0.8700   3rd Qu.:5.000   3rd Qu.:245.0       
 Max.   :1.0000     Max.   :1.0000   Max.   :7.000   Max.   :310.0       
 time_spend_company Work_accident         left        promotion_last_5years
 Min.   : 2.000     Min.   :0.0000   Min.   :0.0000   Min.   :0.00000      
 1st Qu.: 3.000     1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.00000      
 Median : 3.000     Median :0.0000   Median :0.0000   Median :0.00000      
 Mean   : 3.498     Mean   :0.1446   Mean   :0.2381   Mean   :0.02127      
 3rd Qu.: 4.000     3rd Qu.:0.0000   3rd Qu.:0.0000   3rd Qu.:0.00000      
 Max.   :10.000     Max.  

* 범주형 변수 확인

In [ ]:
hr_DF$sales %>% unique()

[1] "sales"       "accounting"  "hr"          "technical"   "support"    
 [6] "management"  "IT"          "product_mng" "marketing"   "RandD"

In [ ]:
hr_DF$salary %>% unique()

[1] "low"    "medium" "high"

In [ ]:
hr_DF$Work_accident %>% unique()

[1] 0 1

In [ ]:
hr_DF$left %>% unique()

[1] 1 0

In [ ]:
hr_DF$promotion_last_5years %>% unique()

[1] 0 1

* mutate를 이용한 데이터 타입 변경(int or chr ---> factor)

In [ ]:
## mutate_at을 이용해 한번에 여러 행값을 넘겨준 후, facter형으로 바꿔준다. 
## levels = 몇개의 종류가 있는지 알려줌 => ex) sales는 총 10가지 종류가 있는데, 그것을 다 범주형으로 바꿔줬다.

hr_DF <- hr_DF %>% mutate_at(`.vars` = c("sales", "salary", "Work_accident", "left", "promotion_last_5years"), `.funs` = as.factor)
hr_DF %>% str()

tibble [14,999 × 10] (S3: tbl_df/tbl/data.frame)
 $ satisfaction_level   : num [1:14999] 0.38 0.8 0.11 0.72 0.37 0.41 0.1 0.92 0.89 0.42 ...
 $ last_evaluation      : num [1:14999] 0.53 0.86 0.88 0.87 0.52 0.5 0.77 0.85 1 0.53 ...
 $ number_project       : int [1:14999] 2 5 7 5 2 2 6 5 5 2 ...
 $ average_montly_hours : int [1:14999] 157 262 272 223 159 153 247 259 224 142 ...
 $ time_spend_company   : int [1:14999] 3 6 4 5 3 3 4 5 5 3 ...
 $ Work_accident        : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ left                 : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ promotion_last_5years: Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ sales                : Factor w/ 10 levels "accounting","hr",..: 8 8 8 8 8 8 8 8 8 8 ...
 $ salary               : Factor w/ 3 levels "high","low","medium": 2 3 3 2 2 2 2 2 2 2 ...
 - attr(*, ".internal.selfref")=<externalptr> 


* left(이직여부)와 salary(연봉수준)의 연관성 검사

In [ ]:
# 이직여부와 연봉수준의 연관성을 검사한다. 
# 교차표로 보여준다.
# cell contents => 자료에 어떤것을 표시했는지 나타내줌 
# 1.N = 관측값, 2. Expected= 기대값, 3.Chi-square= 카이제곱 검정
# Chi^2 =  381.225  -> 카이제곱    
# d.f. =  2  -> 자유도 => (행의갯수-1) * (열의갯수-1)
# p =  1.652087e-83   -> p-value값( 낮은 수치인것을 볼 수 있다. -> 귀무가설 기각 -> 관계가 존재한다.)
# e = 자연로그 -> 2.7xxx 로 생각하면 됨


gmodels::CrossTable(hr_DF$left, hr_DF$salary, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)-> 볼필요 없음


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
             | hr_DF$salary 
  hr_DF$left |      high |       low |    medium | Row Total | 
-------------|-----------|-----------|-----------|-----------|
           0 |      1155 |      5144 |      5129 |     11428 | 
             |   942.492 |  5574.188 |  4911.320 |           | 
             |    47.915 |    33.200 |     9.648 |           | 
             |     0.077 |     0.343 |     0.342 |           | 
-------------|-----------|-----------|-----------|-----------|
           1 |        82 |      2172 |      1317 |      3571 | 
             |   294.508 |  1741.812 |  1534.680 |           | 
             |   153.339 |   106.247 |    30.876 |           | 
             |     0.005 |     0.145 |     0.088 |           | 
-------------|-----------|-----------|

* left(이직여부)와 promotion_last_5years(5년간 승진여부)의 연관성 검사

In [ ]:
# 이직여부와 승진여부를 교차검증한다. 
# p =  3.813123e-14  => 낮은수치임 -> 귀무가설 기각 (관련이 있다.)

gmodels::CrossTable(hr_DF$left, hr_DF$promotion_last_5years, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
             | hr_DF$promotion_last_5years 
  hr_DF$left |         0 |         1 | Row Total | 
-------------|-----------|-----------|-----------|
           0 |     11128 |       300 |     11428 | 
             | 11184.948 |   243.052 |           | 
             |     0.290 |    13.343 |           | 
             |     0.742 |     0.020 |           | 
-------------|-----------|-----------|-----------|
           1 |      3552 |        19 |      3571 | 
             |  3495.052 |    75.948 |           | 
             |     0.928 |    42.702 |           | 
             |     0.237 |     0.001 |           | 
-------------|-----------|-----------|-----------|
Column Total |     14680 |       319 |     14999 | 
-------------|-----------|-----------|-----------|



* left(이직여부)와 Work_accident(사건사고 여부)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$Work_accident, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
             | hr_DF$Work_accident 
  hr_DF$left |         0 |         1 | Row Total | 
-------------|-----------|-----------|-----------|
           0 |      9428 |      2000 |     11428 | 
             |  9775.401 |  1652.599 |           | 
             |    12.346 |    73.029 |           | 
             |     0.629 |     0.133 |           | 
-------------|-----------|-----------|-----------|
           1 |      3402 |       169 |      3571 | 
             |  3054.599 |   516.401 |           | 
             |    39.510 |   233.709 |           | 
             |     0.227 |     0.011 |           | 
-------------|-----------|-----------|-----------|
Column Total |     12830 |      2169 |     14999 | 
-------------|-----------|-----------|-----------|

 
Statis

* left(이직여부)와 sales(부서)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
             | hr_DF$sales 
  hr_DF$left |  accounting |          hr |          IT |  management |   marketing | product_mng |       RandD |       sales |     support |   technical |   Row Total | 
-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
           0 |         563 |         524 |         954 |         539 |         655 |         704 |         666 |        3126 |        1674 |        2023 |       11428 | 
             |     584.391 |     563.057 |     934.873 |     480.008 |     653.725 |     687.250 |     599.629 |    3154.338 |    1698.314 |    2072.415 |             | 
             |       0.783 |       2.709 |       0.391 |      

* promotion_last_5years(5년간 승진여부)와 Work_accident(사건사고 여부)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$promotion_last_5years, hr_DF$Work_accident, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
                            | hr_DF$Work_accident 
hr_DF$promotion_last_5years |         0 |         1 | Row Total | 
----------------------------|-----------|-----------|-----------|
                          0 |     12587 |      2093 |     14680 | 
                            | 12557.130 |  2122.870 |           | 
                            |     0.071 |     0.420 |           | 
                            |     0.839 |     0.140 |           | 
----------------------------|-----------|-----------|-----------|
                          1 |       243 |        76 |       319 | 
                            |   272.870 |    46.130 |           | 
                            |     3.270 |    19.341 |           | 
                            |     0.016 |     0.00

* Work_accident(사건사고 여부)와 sales(부서)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$Work_accident, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
                    | hr_DF$sales 
hr_DF$Work_accident |  accounting |          hr |          IT |  management |   marketing | product_mng |       RandD |       sales |     support |   technical |   Row Total | 
--------------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
                  0 |         671 |         650 |        1063 |         527 |         720 |         770 |         653 |        3553 |        1884 |        2339 |       12830 | 
                    |     656.084 |     632.133 |    1049.564 |     538.896 |     733.925 |     771.562 |     673.192 |    3541.316 |    1906.665 |    2326.662 |             | 
                    |      

* salary(연봉)과 sales(부서)의 연관성 검사


In [ ]:
gmodels::CrossTable(hr_DF$salary, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  14999 

 
             | hr_DF$sales 
hr_DF$salary |  accounting |          hr |          IT |  management |   marketing | product_mng |       RandD |       sales |     support |   technical |   Row Total | 
-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
        high |          74 |          45 |          83 |         225 |          80 |          68 |          51 |         269 |         141 |         201 |        1237 | 
             |      63.256 |      60.947 |     101.193 |      51.957 |      70.761 |      74.390 |      64.906 |     341.435 |     183.830 |     224.324 |             | 
             |       1.825 |       4.173 |       3.271 |     5

#### **B. 예제 2**

---


* https://www.kaggle.com/c/titanic/data

**<kaggle의 타이타닉 data>**

  * survived : 생존=1, 죽음=0
  * pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
  * sibsp : 함께 탑승한 형제 또는 배우자 수
  * parch : 함께 탑승한 부모 또는 자녀 수
  * ticket : 티켓 번호
  * cabin : 선실 번호
  * embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

In [ ]:
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/train.csv") %>%
 as_tibble()

In [ ]:
str(DF)

tibble [891 × 12] (S3: tbl_df/tbl/data.frame)
 $ PassengerId: int [1:891] 1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : int [1:891] 0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass     : int [1:891] 3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : chr [1:891] "Braund, Mr. Owen Harris" "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" "Heikkinen, Miss. Laina" "Futrelle, Mrs. Jacques Heath (Lily May Peel)" ...
 $ Sex        : chr [1:891] "male" "female" "female" "female" ...
 $ Age        : num [1:891] 22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp      : int [1:891] 1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int [1:891] 0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : chr [1:891] "A/5 21171" "PC 17599" "STON/O2. 3101282" "113803" ...
 $ Fare       : num [1:891] 7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : chr [1:891] "" "C85" "" "C123" ...
 $ Embarked   : chr [1:891] "S" "C" "S" "S" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [ ]:
DF %>% summary()

  PassengerId       Survived          Pclass          Name          
 Min.   :  1.0   Min.   :0.0000   Min.   :1.000   Length:891        
 1st Qu.:223.5   1st Qu.:0.0000   1st Qu.:2.000   Class :character  
 Median :446.0   Median :0.0000   Median :3.000   Mode  :character  
 Mean   :446.0   Mean   :0.3838   Mean   :2.309                     
 3rd Qu.:668.5   3rd Qu.:1.0000   3rd Qu.:3.000                     
 Max.   :891.0   Max.   :1.0000   Max.   :3.000                     
                                                                    
     Sex                 Age            SibSp           Parch       
 Length:891         Min.   : 0.42   Min.   :0.000   Min.   :0.0000  
 Class :character   1st Qu.:20.12   1st Qu.:0.000   1st Qu.:0.0000  
 Mode  :character   Median :28.00   Median :0.000   Median :0.0000  
                    Mean   :29.70   Mean   :0.523   Mean   :0.3816  
                    3rd Qu.:38.00   3rd Qu.:1.000   3rd Qu.:0.0000  
                    Max.   :80.00 

* 범주형 변수 확인

In [ ]:
DF$Survived %>% unique()

[1] 0 1

In [ ]:
DF$Pclass %>% unique()

[1] 3 1 2

In [ ]:
DF$Sex %>% unique()

[1] "male"   "female"

In [ ]:
DF$Ticket %>% unique()

[1] "A/5 21171"          "PC 17599"           "STON/O2. 3101282"  
  [4] "113803"             "373450"             "330877"            
  [7] "17463"              "349909"             "347742"            
 [10] "237736"             "PP 9549"            "113783"            
 [13] "A/5. 2151"          "347082"             "350406"            
 [16] "248706"             "382652"             "244373"            
 [19] "345763"             "2649"               "239865"            
 [22] "248698"             "330923"             "113788"            
 [25] "347077"             "2631"               "19950"             
 [28] "330959"             "349216"             "PC 17601"          
 [31] "PC 17569"           "335677"             "C.A. 24579"        
 [34] "PC 17604"           "113789"             "2677"              
 [37] "A./5. 2152"         "345764"             "2651"              
 [40] "7546"               "11668"              "349253"            
 [43] "SC/Paris 2123"      "330958"             "S.C./A.4. 23567"   
 [46] "370371"             "14311"              "2662"              
 [49] "349237"             "3101295"            "A/4. 39886"        
 [52] "PC 17572"           "2926"               "113509"            
 [55] "19947"              "C.A. 31026"         "2697"              
 [58] "C.A. 34651"         "CA 2144"            "2669"              
 [61] "113572"             "36973"              "347088"            
 [64] "PC 17605"           "2661"               "C.A. 29395"        
 [67] "S.P. 3464"          "3101281"            "315151"            
 [70] "C.A. 33111"         "S.O.C. 14879"       "2680"              
 [73] "1601"               "348123"             "349208"            
 [76] "374746"             "248738"             "364516"            
 [79] "345767"             "345779"             "330932"            
 [82] "113059"             "SO/C 14885"         "3101278"           
 [85] "W./C. 6608"         "SOTON/OQ 392086"    "343275"            
 [88] "343276"             "347466"             "W.E.P. 5734"       
 [91] "C.A. 2315"          "364500"             "374910"            
 [94] "PC 17754"           "PC 17759"           "231919"            
 [97] "244367"             "349245"             "349215"            
[100] "35281"              "7540"               "3101276"           
[103] "349207"             "343120"             "312991"            
[106] "349249"             "371110"             "110465"            
[109] "2665"               "324669"             "4136"              
[112] "2627"               "STON/O 2. 3101294"  "370369"            
[115] "PC 17558"           "A4. 54510"          "27267"             
[118] "370372"             "C 17369"            "2668"              
[121] "347061"             "349241"             "SOTON/O.Q. 3101307"
[124] "A/5. 3337"          "228414"             "C.A. 29178"        
[127] "SC/PARIS 2133"      "11752"              "7534"              
[130] "PC 17593"           "2678"               "347081"            
[133] "STON/O2. 3101279"   "365222"             "231945"            
[136] "C.A. 33112"         "350043"             "230080"            
[139] "244310"             "S.O.P. 1166"        "113776"            
[142] "A.5. 11206"         "A/5. 851"           "Fa 265302"         
[145] "PC 17597"           "35851"              "SOTON/OQ 392090"   
[148] "315037"             "CA. 2343"           "371362"            
[151] "C.A. 33595"         "347068"             "315093"            
[154] "363291"             "113505"             "PC 17318"          
[157] "111240"             "STON/O 2. 3101280"  "17764"             
[160] "350404"             "4133"               "PC 17595"          
[163] "250653"             "LINE"               "SC/PARIS 2131"     
[166] "230136"             "315153"             "113767"            
[169] "370365"             "111428"             "364849"            
[172] "349247"             "234604"             "28424"             


In [ ]:
DF$Embarked %>% unique()

[1] "S" "C" "Q" ""

In [ ]:
DF <- DF %>% 
 select(-PassengerId, -Name, -Cabin, -Ticket) %>% mutate_at(c("Pclass","Survived","Sex","Embarked"), factor)
str(DF)

tibble [891 × 8] (S3: tbl_df/tbl/data.frame)
 $ Survived: Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 1 2 2 ...
 $ Pclass  : Factor w/ 3 levels "1","2","3": 3 1 3 1 3 3 1 3 3 2 ...
 $ Sex     : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age     : num [1:891] 22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp   : int [1:891] 1 1 0 1 0 0 0 3 0 1 ...
 $ Parch   : int [1:891] 0 0 0 0 0 0 0 1 2 0 ...
 $ Fare    : num [1:891] 7.25 71.28 7.92 53.1 8.05 ...
 $ Embarked: Factor w/ 4 levels "","C","Q","S": 4 2 4 4 4 3 4 4 4 2 ...
 - attr(*, ".internal.selfref")=<externalptr> 


* Survived(생존여부)와 Pclass(승객 등급)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Pclass, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  891 

 
             | DF$Pclass 
 DF$Survived |         1 |         2 |         3 | Row Total | 
-------------|-----------|-----------|-----------|-----------|
           0 |        80 |        97 |       372 |       549 | 
             |   133.091 |   113.374 |   302.535 |           | 
             |    21.178 |     2.365 |    15.950 |           | 
             |     0.090 |     0.109 |     0.418 |           | 
-------------|-----------|-----------|-----------|-----------|
           1 |       136 |        87 |       119 |       342 | 
             |    82.909 |    70.626 |   188.465 |           | 
             |    33.997 |     3.796 |    25.603 |           | 
             |     0.153 |     0.098 |     0.134 |           | 
-------------|-----------|-----------|-----

* Survived(생존여부)와 Sex(성별)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Sex, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  891 

 
             | DF$Sex 
 DF$Survived |    female |      male | Row Total | 
-------------|-----------|-----------|-----------|
           0 |        81 |       468 |       549 | 
             |   193.475 |   355.525 |           | 
             |    65.386 |    35.583 |           | 
             |     0.091 |     0.525 |           | 
-------------|-----------|-----------|-----------|
           1 |       233 |       109 |       342 | 
             |   120.525 |   221.475 |           | 
             |   104.962 |    57.120 |           | 
             |     0.262 |     0.122 |           | 
-------------|-----------|-----------|-----------|
Column Total |       314 |       577 |       891 | 
-------------|-----------|-----------|-----------|

 
Statistics for All Ta

* Survived(생존여부)와 Embarked(탑승 장소)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Embarked, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  891 

 
             | DF$Embarked 
 DF$Survived |           |         C |         Q |         S | Row Total | 
-------------|-----------|-----------|-----------|-----------|-----------|
           0 |         0 |        75 |        47 |       427 |       549 | 
             |     1.232 |   103.515 |    47.444 |   396.808 |           | 
             |     1.232 |     7.855 |     0.004 |     2.297 |           | 
             |     0.000 |     0.084 |     0.053 |     0.479 |           | 
-------------|-----------|-----------|-----------|-----------|-----------|
           1 |         2 |        93 |        30 |       217 |       342 | 
             |     0.768 |    64.485 |    29.556 |   247.192 |           | 
             |     1.978 |    12.609 |     0.007 |     3.688 

* Pclass(승객 등급)과 Sex(성별)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Pclass, DF$Sex, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)


 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  891 

 
             | DF$Sex 
   DF$Pclass |    female |      male | Row Total | 
-------------|-----------|-----------|-----------|
           1 |        94 |       122 |       216 | 
             |    76.121 |   139.879 |           | 
             |     4.199 |     2.285 |           | 
             |     0.105 |     0.137 |           | 
-------------|-----------|-----------|-----------|
           2 |        76 |       108 |       184 | 
             |    64.844 |   119.156 |           | 
             |     1.919 |     1.044 |           | 
             |     0.085 |     0.121 |           | 
-------------|-----------|-----------|-----------|
           3 |       144 |       347 |       491 | 
             |   173.035 |   317.965 |           | 
             |     4.87

* Sex(성별)과 Embarked(탑승 장소)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Sex, DF$Embarked, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
|              Expected N |
| Chi-square contribution |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  891 

 
             | DF$Embarked 
      DF$Sex |           |         C |         Q |         S | Row Total | 
-------------|-----------|-----------|-----------|-----------|-----------|
      female |         2 |        73 |        36 |       203 |       314 | 
             |     0.705 |    59.205 |    27.136 |   226.954 |           | 
             |     2.380 |     3.214 |     2.896 |     2.528 |           | 
             |     0.002 |     0.082 |     0.040 |     0.228 |           | 
-------------|-----------|-----------|-----------|-----------|-----------|
        male |         0 |        95 |        41 |       441 |       577 | 
             |     1.295 |   108.795 |    49.864 |   417.046 |           | 
             |     1.295 |     1.749 |     1.576 |     1.376 

### **<데이터베이스 연동>**

* 데이터베이스란?
  - 데이터베이스는 체계화된 데이터의 모임이다. 즉, 작성된 목록으로서 여러 응용시스템들의 통합된 정보들을 저장하여 운영할 수 있는 공용데이터들의 묶음이다. 여러 사람들이 공유하고 사용할 목적으로 통합 관리되는 정보의 집합이다. 논리적으로 연관된 하나 이상의 자료의 모음으로 그 내용을 고도로 구조화함으로써 검색과 갱신의 효율화를 꾀한 것이다. 즉, 몇 개의 자료 파일을 조직적으로 통합하여 자료 항목의 중복을 없애고 자료를 구조화하여 기억시켜 놓은 자료의 집합체라고 할 수 있다.

출처: https://neos518.tistory.com/122 [As I've always been]



In [ ]:
#install.packages("tidyverse")
library(tidyverse)

#### **A. SQLite**

https://ehdvudee.tistory.com/23

---


In [ ]:
## 두가지 패키지를 설치함으로써 데이터베이스를 사용할수 있다.

install.packages(c("DBI", "RSQLite"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘plogr’




In [ ]:
library(DBI)
library(RSQLite)

* connection 생성

In [ ]:
# 현재 폴더가 없기 때문에 오류남

con <- RSQLite::dbConnect(drv = SQLite(), dbname = "/sqlite/height_weight.sqlite")

ERROR: ignored

In [ ]:
# 현재 디렉토리
# 현재 저장되는 공간 -> content 밑에 squlite 폴더 생성

getwd()

[1] "/content"

In [ ]:
# 위에처럼 파일 만들어도 되고, dir을 이용해서 폴더 만들어줘도 된다.
# 워킹디렉토리에 폴더 생성

dir.create("sqlite")

In [ ]:
# 어떤 폴더가 있는지 보여줌

list.dirs()

[1] "."                         "./.config"                
[3] "./.config/configurations"  "./.config/logs"           
[5] "./.config/logs/2020.11.06" "./.ipynb_checkpoints"     
[7] "./sample_data"             "./sqlite"                 
[9] "./squlit"

In [ ]:
# sqlite 폴더가 없음. 따라서 생성
# sqlite밑에 (height_weight)파일을 생성
# 이 한줄로 R에서 데이터베이스를 사용할 수 있게되는거임. -> 파일을 생성해야함

con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/height_weight.sqlite")

In [ ]:
# RSQLite::dbDisconnect(con) # connection 끊기

In [ ]:

list.files("sqlite/")

[1] "height_weight.sqlite" "seoul_bike.sqlite"

In [ ]:
dbGetInfo(dbObj = con)

$db.version
[1] "3.33.0"

$dbname
[1] "sqlite/height_weight.sqlite"

$username
[1] NA

$host
[1] NA

$port
[1] NA

* 테이블 생성하기

```
ID : 사람의 고유한 ID(primary key), integer
HEIGHT : 키(cm), numeric
WEIGHT : 몸무게(kg), numeric

### PRIMARY KEY ??? --> https://jins-dev.tistory.com/entry/RDB-MySQL-%EC%97%90%EC%84%9C%EC%9D%98-Primary-Key-%EB%9E%80
```



In [ ]:
# 테이블이 있는지 확인 -> 아무 테이블도 없는것으로 나옴

dbListTables(con)

character(0)

- ``dbSendQuery``는 DB에 쿼리를 보낼 때 사용. 단, 데이터를 가지고 오지는 못함. 

- 또한, ``dbSendQuery``의 경우 ``dbClearResult`` 함수를 이용하여 보낸 쿼리의 처리결과를 제거해주어야 함

In [ ]:
# 테이블 생성
# dbSendQuery함수를 이용해 DB에 쿼리를 보낸다. -> 댠, 데이터를 가지고 오지는 못한다.
# 따라서 쿼리만 보낼 수 있다.

table_query <- dbSendQuery(conn = con, 
                           statement = "CREATE TABLE HW_table(
                                          ID INTEGER PRIMARY KEY,
                                          HEIGHT NUMERIC,
                                          WEIGHT NUMERIC)")

In [ ]:
# 쿼리의 결과를 제거해줘야함

dbClearResult(table_query)

* 생성된 table 확인

In [ ]:
# 테이블이 만들어진것을 확인할 수 잇다.

dbListTables(con)

[1] "HW_table"

In [ ]:
# 테이블 이름을 주면 있는지 없는지 파악할 수 있다.


dbExistsTable(con, "HW_table") # False 일 경우, 생성이 안 된 것.

[1] TRUE

In [ ]:
dbListFields(con, "HW_table")

[1] "ID"     "HEIGHT" "WEIGHT"

* 생성된 `HW_table`에 데이터 넣기 - 한번에 1행(row)

In [ ]:
# dbSendQuery = 한번에 1개 행씩 넣어주는 방법
# dbSendQuery은 한번에 1개행만 넣을 수 있다.

row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (1, 183, 85)")
dbClearResult(row_add_query)#

In [ ]:
# error 오류남

# ID값이 중복되서 들어가서 -> primary key라서

# row_add_query  <- dbSendQuery(con, 
#                                   "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (1, 182, 67)")
# dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (2, 170, 68)")
dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (3, 165, 65)")
dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (5, 165, 65)")
dbClearResult(row_add_query)

* 테이블에서 데이터 가져오기

In [ ]:
# dbGetQuery = 테이블에서 데이터를 가져올 수 있다.
# 데이터 프레임의 형태로 반환됨


dbGetQuery(con, "SELECT * FROM HW_table")

ID,HEIGHT,WEIGHT
<int>,<int>,<int>
1,183,85
2,170,68
3,165,65
5,165,65


In [ ]:
# 데이터 베이스로부터 받아온 정보들을 DF에다가 저장


DF <- dbGetQuery(con, "SELECT * FROM HW_table")
DF %>% str()

'data.frame':	4 obs. of  3 variables:
 $ ID    : int  1 2 3 5
 $ HEIGHT: int  183 170 165 165
 $ WEIGHT: int  85 68 65 65


In [ ]:
# lm=회귀분석 

lm(HEIGHT ~ WEIGHT, data = DF) %>% summary()


Call:
lm(formula = HEIGHT ~ WEIGHT, data = DF)

Residuals:
      1       2       3       4 
-0.2493  1.6621 -0.7064 -0.7064 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)   
(Intercept) 108.69196    5.92035   18.36  0.00295 **
WEIGHT        0.87715    0.08311   10.55  0.00886 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.383 on 2 degrees of freedom
Multiple R-squared:  0.9824,	Adjusted R-squared:  0.9735 
F-statistic: 111.4 on 1 and 2 DF,  p-value: 0.008858


In [ ]:
dbGetQuery(con, "SELECT ID, HEIGHT FROM HW_table")

ID,HEIGHT
<int>,<int>
1,183
2,170
3,165
5,165


In [ ]:
# 조건을 걸어서 가져옴

dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 180")

ID,HEIGHT,WEIGHT
<int>,<int>,<int>
1,183,85


In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 180 or WEIGHT >= 68")

ID,HEIGHT,WEIGHT
<int>,<int>,<int>
1,183,85
2,170,68


* 생성된 `HW_table`에 데이터 넣기 - 한번에 여러행

In [ ]:
# dbSendQuery는 데이터를 넣는데 오래걸리기 때문에 한번에 넣어주는 방법이 필요
# 한번에 넣어주는 방법시작
# R에서 DF를 만들고, DB 테이블에 한번에 저장
# 복원추출로 500개의 데이터를 뽑아서 height와 weight 에 넣어줌

temp <- tibble(height = sample(150:210, 500, replace = T), weight = sample(45:120, 500, replace = T)) # 복원 추출(중복 허용)
temp %>% show()

# A tibble: 500 x 2
   height weight
    <int>  <int>
 1    195    103
 2    204     46
 3    184     80
 4    164     85
 5    178     45
 6    163     47
 7    168    116
 8    153     91
 9    200    101
10    158     75
# ... with 490 more rows


In [ ]:
# 해당 Table의 column 명과 동일해야 함. error(에러남)


# dbWriteTable(con, "HW_table", temp, row.names = F, append = T)

In [ ]:
# append = 우리가 사용하고 있는 테이블의 데이터에 추가적으로 데이터를 넣어준다.
#  row.names = F => T해주게 되면 우리가 만든 행보다 더 만들어질 수 있기 때문에 원치않는 정보가 들어갈 수있음.
# 데이터베이스에 만든 colume명과 DF의 열 이름이 동일해야 한다.

temp <- tibble(HEIGHT = sample(150:210, 500, replace = T), WEIGHT = sample(45:120, 500, replace = T)) # 복원 추출(중복 허용)
temp %>% show()

# A tibble: 500 x 2
   HEIGHT WEIGHT
    <int>  <int>
 1    157     85
 2    195     97
 3    189     81
 4    195    106
 5    182    103
 6    173     87
 7    175     71
 8    173     49
 9    161     50
10    168     55
# ... with 490 more rows


In [ ]:
# 해당 Table의 column 명과 동일해야 함.
dbWriteTable(con, "HW_table", temp, row.names = F, append = T)

In [ ]:
# !!!!!중요!!!!!!
# ID의 경우 PRIMARY KEY이므로 자동적으로 중복되지 않은 값이 채워짐.
# 데이터를 넣어줄 때 ID는 넣어주지 않았다. -> primary key라서 알아서 값이 채워진다.


dbGetQuery(con, "SELECT * FROM HW_table")

ID,HEIGHT,WEIGHT
<int>,<int>,<int>
1,183,85
2,170,68
3,165,65
5,165,65
6,209,64
7,187,97
8,183,107
9,206,61
10,197,104


* 테이블에 있는 데이터 삭제하기(DELETE FROM ~ WHERE)

In [ ]:
# 테이블에 있는 데이터 삭제

query <- dbSendQuery(con, "DELETE FROM HW_table WHERE HEIGHT >= 200")
dbClearResult(query)

In [ ]:
# 삭제가 되었기 때문에 존재하지 않는다고 나온다.

dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 200")

ID,HEIGHT,WEIGHT
<int>,<dbl>,<dbl>


In [ ]:
# 전체 데이터 삭제(table 삭제가 아님!!)
# 테이블에 속한 데이터를 모두다 삭제 -> 값만 지워지고 구조는 남아있음
# 구조까지 지우고 싶으면 drop table하면됨

query <- dbSendQuery(con, "DELETE FROM HW_table")
dbClearResult(query)

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table")

ID,HEIGHT,WEIGHT
<int>,<dbl>,<dbl>


* table 삭제(DROP TABLE)

In [ ]:
query <- dbSendQuery(con, "DROP TABLE HW_table")
dbClearResult(query)

In [ ]:
# 에러가 뜬다. 테이블이 완전히 삭제되었기 때문에

# dbGetQuery(con, "SELECT * FROM HW_table")

In [ ]:
dbListTables(con, "HW_table")

character(0)

In [ ]:
# 존재하지 않기 때문에 false리턴한다.

dbExistsTable(con, "HW_table")

[1] FALSE

* connection 종료

In [ ]:
# DB 끝내는 방법 -> 데이터베이스를 사용할 수 없게됨

dbDisconnect(con)

* SQLite의 이식성(!?)

In [ ]:
# Lab 9 참고
#https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=dateDown&view=table
#https://datascience.stackexchange.com/questions/8922/removing-strings-after-a-certain-character-in-a-given-text
#https://stackoverflow.com/questions/5992082/how-to-remove-all-whitespace-from-a-string

Sys.setlocale('LC_ALL','C') # 이부분 무시해도 됨.
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/SeoulBikeData.csv", sep = ",", stringsAsFactors = F) %>%
 as_tibble()

colnames(DF) <- colnames(DF) %>% lapply(X = ., FUN = function(value){
  stringr::str_remove(string = value, pattern = "\\(.*") %>%
  stringr::str_replace_all(string = ., pattern = "[[:space:]]", replace = "_")
}) %>% unlist()

DF %>% str()

[1] "LC_CTYPE=C;LC_NUMERIC=C;LC_TIME=C;LC_COLLATE=C;LC_MONETARY=C;LC_MESSAGES=en_US.UTF-8;LC_PAPER=en_US.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=en_US.UTF-8;LC_IDENTIFICATION=C"

tibble [8,760 x 14] (S3: tbl_df/tbl/data.frame)
 $ Date                 : chr [1:8760] "01/12/2017" "01/12/2017" "01/12/2017" "01/12/2017" ...
 $ Rented_Bike_Count    : int [1:8760] 254 204 173 107 78 100 181 460 930 490 ...
 $ Hour                 : int [1:8760] 0 1 2 3 4 5 6 7 8 9 ...
 $ Temperature          : num [1:8760] -5.2 -5.5 -6 -6.2 -6 -6.4 -6.6 -7.4 -7.6 -6.5 ...
 $ Humidity             : int [1:8760] 37 38 39 40 36 37 35 38 37 27 ...
 $ Wind_speed_          : num [1:8760] 2.2 0.8 1 0.9 2.3 1.5 1.3 0.9 1.1 0.5 ...
 $ Visibility_          : int [1:8760] 2000 2000 2000 2000 2000 2000 2000 2000 2000 1928 ...
 $ Dew_point_temperature: num [1:8760] -17.6 -17.6 -17.7 -17.6 -18.6 -18.7 -19.5 -19.3 -19.8 -22.4 ...
 $ Solar_Radiation_     : num [1:8760] 0 0 0 0 0 0 0 0 0.01 0.23 ...
 $ Rainfall             : num [1:8760] 0 0 0 0 0 0 0 0 0 0 ...
 $ Snowfall_            : num [1:8760] 0 0 0 0 0 0 0 0 0 0 ...
 $ Seasons              : chr [1:8760] "Winter" "Winter" "Winter" "Winter" ...

In [ ]:
# 테이블을 만든다 -> content에 sqlite폴더가 만들어져 있어야함.

con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/seoul_bike.sqlite")

In [ ]:
# DF에 저장되어있는 값을 테이블에 넣어준다.

dbWriteTable(con, "seoul_bike", DF, row.names = F, append = F)

In [ ]:
#테이블 확인

dbListTables(con)

[1] "seoul_bike"

In [ ]:
# 필드 확인
dbListFields(con, "seoul_bike")

[1] "Date"                  "Rented_Bike_Count"     "Hour"                 
 [4] "Temperature"           "Humidity"              "Wind_speed_"          
 [7] "Visibility_"           "Dew_point_temperature" "Solar_Radiation_"     
[10] "Rainfall"              "Snowfall_"             "Seasons"              
[13] "Holiday"               "Functioning_Day"

In [ ]:
dbGetQuery(con, "SELECT * FROM seoul_bike") %>%
 as_tibble() %>% str()

tibble [8,760 x 14] (S3: tbl_df/tbl/data.frame)
 $ Date                 : chr [1:8760] "01/12/2017" "01/12/2017" "01/12/2017" "01/12/2017" ...
 $ Rented_Bike_Count    : int [1:8760] 254 204 173 107 78 100 181 460 930 490 ...
 $ Hour                 : int [1:8760] 0 1 2 3 4 5 6 7 8 9 ...
 $ Temperature          : num [1:8760] -5.2 -5.5 -6 -6.2 -6 -6.4 -6.6 -7.4 -7.6 -6.5 ...
 $ Humidity             : int [1:8760] 37 38 39 40 36 37 35 38 37 27 ...
 $ Wind_speed_          : num [1:8760] 2.2 0.8 1 0.9 2.3 1.5 1.3 0.9 1.1 0.5 ...
 $ Visibility_          : int [1:8760] 2000 2000 2000 2000 2000 2000 2000 2000 2000 1928 ...
 $ Dew_point_temperature: num [1:8760] -17.6 -17.6 -17.7 -17.6 -18.6 -18.7 -19.5 -19.3 -19.8 -22.4 ...
 $ Solar_Radiation_     : num [1:8760] 0 0 0 0 0 0 0 0 0.01 0.23 ...
 $ Rainfall             : num [1:8760] 0 0 0 0 0 0 0 0 0 0 ...
 $ Snowfall_            : num [1:8760] 0 0 0 0 0 0 0 0 0 0 ...
 $ Seasons              : chr [1:8760] "Winter" "Winter" "Winter" "Winter" ...

In [ ]:
dbDisconnect(con)

#### **B. dbplyr**

https://dbplyr.tidyverse.org/index.html

https://dbplyr.tidyverse.org/articles/sql-translation.html

```
dbplyr is the database backend for dplyr. 
It allows you to use remote database tables as if they are in-memory data frames by automatically converting dplyr code into SQL.
```

---


In [ ]:
# dbplyr을 사용해서 데이터베이스를 R언어로 사용할 수 있다.
# 폴더 생성 -> DB 이용할 수 있게함

con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/seoul_bike.sqlite")

* copy_to를 local 데이터를 database에 복제

In [ ]:
dbListTables(con)

[1] "iris"         "seoul_bike"   "sqlite_stat1" "sqlite_stat4"

In [ ]:
# dplyr을 이용해 데이터베이스에 iris 데이터를 넣어줄 수 잇다.
# local 데이터를 데이터베이스에 복제함
# iris 테이블이 생성된것을 볼 수 있다.

dplyr::copy_to(con, iris)

In [ ]:
dbListTables(con)

[1] "iris"         "seoul_bike"   "sqlite_stat1" "sqlite_stat4"

* tbl을 이용한 database의 table 사용(dataframe)

In [ ]:
# tbl함수를 이용하면 데이터베이스에 있는 데이터를 DF에서 사용하는 것처럼 사용할 수 있다.
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite] -> 데이터베이스에서 온것을 알 수 있다.

iris_2 <- tbl(con, "iris")#데이터베이스에 있는 테이블명
iris_2

# Source:   table<iris> [?? x 5]
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite]
   Sepal.Length Sepal.Width Petal.Length Petal.Width Species
          <dbl>       <dbl>        <dbl>       <dbl> <chr>  
 1          5.1         3.5          1.4         0.2 setosa 
 2          4.9         3            1.4         0.2 setosa 
 3          4.7         3.2          1.3         0.2 setosa 
 4          4.6         3.1          1.5         0.2 setosa 
 5          5           3.6          1.4         0.2 setosa 
 6          5.4         3.9          1.7         0.4 setosa 
 7          4.6         3.4          1.4         0.3 setosa 
 8          5           3.4          1.5         0.2 setosa 
 9          4.4         2.9          1.4         0.2 setosa 
10          4.9         3.1          1.5         0.1 setosa 
# ... with more rows

In [ ]:
iris %>% show()

    Sepal.Length Sepal.Width Petal.Length Petal.Width    Species
1            5.1         3.5          1.4         0.2     setosa
2            4.9         3.0          1.4         0.2     setosa
3            4.7         3.2          1.3         0.2     setosa
4            4.6         3.1          1.5         0.2     setosa
5            5.0         3.6          1.4         0.2     setosa
6            5.4         3.9          1.7         0.4     setosa
7            4.6         3.4          1.4         0.3     setosa
8            5.0         3.4          1.5         0.2     setosa
9            4.4         2.9          1.4         0.2     setosa
10           4.9         3.1          1.5         0.1     setosa
11           5.4         3.7          1.5         0.2     setosa
12           4.8         3.4          1.6         0.2     setosa
13           4.8         3.0          1.4         0.1     setosa
14           4.3         3.0          1.1         0.1     setosa
15           5.8         

In [ ]:
seoul_bike_2 <- tbl(con, "seoul_bike")
seoul_bike_2

# Source:   table<seoul_bike> [?? x 14]
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite]
   Date  Rented_Bike_Cou~  Hour Temperature Humidity Wind_speed_ Visibility_
   <chr>            <int> <int>       <dbl>    <int>       <dbl>       <int>
 1 01/1~              254     0        -5.2       37         2.2        2000
 2 01/1~              204     1        -5.5       38         0.8        2000
 3 01/1~              173     2        -6         39         1          2000
 4 01/1~              107     3        -6.2       40         0.9        2000
 5 01/1~               78     4        -6         36         2.3        2000
 6 01/1~              100     5        -6.4       37         1.5        2000
 7 01/1~              181     6        -6.6       35         1.3        2000
 8 01/1~              460     7        -7.4       38         0.9        2000
 9 01/1~              930     8        -7.6       37         1.1        2000
10 01/1~              490     9        -6.5       2

* dplyr 문법을 이용한 SQL 번역

In [ ]:
# 데이터베이스에 있는 값들을 R언어로 사용할 수 있다.
# dplyr 에서 자동으로 SQL 언어로 번역해서 일을 처리해줌
# 하지만 collect()를 써주지 않는다면 실제 데이터가 되지않고, 출력만 해주게 된다 -> 실제 데이터는 변경되지 않음

seoul_bike_2 %>% select(Date, Hour)

# Source:   lazy query [?? x 2]
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite]
   Date        Hour
   <chr>      <int>
 1 01/12/2017     0
 2 01/12/2017     1
 3 01/12/2017     2
 4 01/12/2017     3
 5 01/12/2017     4
 6 01/12/2017     5
 7 01/12/2017     6
 8 01/12/2017     7
 9 01/12/2017     8
10 01/12/2017     9
# ... with more rows

In [ ]:
seoul_bike_2 %>% filter(Humidity >= 40)

# Source:   lazy query [?? x 14]
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite]
   Date  Rented_Bike_Cou~  Hour Temperature Humidity Wind_speed_ Visibility_
   <chr>            <int> <int>       <dbl>    <int>       <dbl>       <int>
 1 01/1~              107     3        -6.2       40         0.9        2000
 2 01/1~              484    16         1.2       54         4.2         793
 3 01/1~              555    17         0.8       58         1.6        2000
 4 01/1~              862    18         0.6       66         1.4        2000
 5 01/1~              600    19         0         77         1.7        2000
 6 01/1~              426    20        -0.3       79         1.5        1913
 7 01/1~              405    21        -0.8       81         0.8        1687
 8 01/1~              398    22        -0.9       83         1.5        1380
 9 01/1~              323    23        -1.3       84         1          1265
10 02/1~              328     0        -1.8       87      

In [ ]:
seoul_bike_2 %>% group_by(Date) %>% summarise(daily_humidity = mean(Humidity, na.rm = T))

# Source:   lazy query [?? x 2]
# Database: sqlite 3.33.0 [/content/sqlite/seoul_bike.sqlite]
   Date       daily_humidity
   <chr>               <dbl>
 1 01/01/2018           39.3
 2 01/02/2018           44  
 3 01/03/2018           64.2
 4 01/04/2018           68.9
 5 01/05/2018           72.8
 6 01/06/2018           50.1
 7 01/07/2018           93.5
 8 01/08/2018           55.8
 9 01/09/2018           61.2
10 01/10/2018           54.2
# ... with more rows

* Show Query

In [ ]:
daily_h <- seoul_bike_2 %>% filter(Humidity >= 40) %>% 
 group_by(Date) %>% summarise(daily_humidity = mean(Humidity, na.rm = T))

In [ ]:
# R언어가 SQL문인 쿼리으로 바뀐것을 볼 수 있다.

show_query(daily_h)

<SQL>
SELECT `Date`, AVG(`Humidity`) AS `daily_humidity`
FROM `seoul_bike`
WHERE (`Humidity` >= 40.0)
GROUP BY `Date`


In [ ]:
explain(daily_h)

<SQL>
SELECT `Date`, AVG(`Humidity`) AS `daily_humidity`
FROM `seoul_bike`
WHERE (`Humidity` >= 40.0)
GROUP BY `Date`

<PLAN>
  id parent notused                       detail
1  6      0       0        SCAN TABLE seoul_bike
2 10      0       0 USE TEMP B-TREE FOR GROUP BY


* Collect Data

```
Now, some interesting facts. When working with databases, dplyr never pulls data into R unless you explicitly ask for it. 
In the previous example, dplyr will not do anything until you ask for the mileages data. 

It generates the SQL and only pulls down a few rows when you try to print mileages. So how do we pull all the data and store it for further analysis? 
collect() will pull all the data and store it in a tibble and you can use it for any further analysis.
```

In [ ]:
# collect()를 써주지 않는다면 실제 데이터에 영향을 줄 수 없다.


daily_h %>% collect() %>% show()

# A tibble: 361 x 2
   Date       daily_humidity
   <chr>               <dbl>
 1 01/01/2018           47.6
 2 01/02/2018           54.9
 3 01/03/2018           71.7
 4 01/04/2018           68.9
 5 01/05/2018           72.8
 6 01/06/2018           60.9
 7 01/07/2018           93.5
 8 01/08/2018           60.4
 9 01/09/2018           61.2
10 01/10/2018           55.0
# ... with 351 more rows


#### **C. Google Cloud Platform(GCP)를 이용한 DB 서버 생성 및 이용**

https://blog-rogan.tistory.com/entry/Cloud-Platform%EC%9D%B4%EB%9E%80

https://cloud.google.com/free/docs/gcp-free-tier?hl=ko

---


In [ ]:

system("sudo apt-get install libmysql++-dev")

In [ ]:
install.packages("RMariaDB")

Installing package into '/usr/local/lib/R/site-library'
(as 'lib' is unspecified)



In [ ]:
#dbDisconnect(con)

In [ ]:
con <- dbConnect(RMariaDB::MariaDB(), user = "root", password = "spark123!",
                          host = "34.123.12.131", dbname = "test")

* 예제 데이터

In [ ]:
#install.packages("nycflights13")
nycflights13::flights %>% str()

In [ ]:
copy_to(con, nycflights13::flights, "flights")

* 예제 데이터

In [ ]:
Sys.setlocale('LC_ALL','C') # 이부분 무시해도 됨.
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/SeoulBikeData.csv", sep = ",", stringsAsFactors = F) %>%
 as_tibble()

colnames(DF) <- colnames(DF) %>% lapply(X = ., FUN = function(value){
  stringr::str_remove(string = value, pattern = "\\(.*") %>%
  stringr::str_replace_all(string = ., pattern = "[[:space:]]", replace = "_")
}) %>% unlist()

In [ ]:
copy_to(con, DF, "seoul_bike")

- - -

## **실습 과제 <11주차 과제 - 100점>**

### **(1) 아래의 조건에 맞는 ```학번_이름.sqlite 파일```을 생성한 후, 한림SmartLEAD에 제출할 것(ex.D20514_이진우.sqlite)**
### **(2) 제출 데드라인 : 2020.11.9(월요일) ~ 2020.11.15(일요일) 23:59 까지**


- - -


### **1. SQLite를 이용하여 학번_이름.sqlite 데이터베이스를 생성**

### **2. dbplyr를 이용하여 학번_이름(영어)인 테이블 생성 (ex. D20514_jinoo)**

```
<테이블의 구조>

ID : 자신의 학번
GRADE : 자신의 학년
NAME : 자신의 이름
RESIDENCE : 자신의 거주 지역
```


### **3. dbplyr를 이용하여 각 열에 맞는 데이터 삽입(1행만)**



* 예시 코드

In [14]:
install.packages(c("DBI", "RSQLite", "tidyverse", "dbplyr"))
library(DBI)
library(RSQLite)
library(tidyverse)
library(dbplyr)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [15]:
# test 폴더 생성

getwd()
dir.create("test")#dir함수 사용해서 폴더 생성
list.dirs()# 무슨 폴더 있는지 알려줌

[1] "/content"

[1] "."                         "./.config"                
[3] "./.config/configurations"  "./.config/logs"           
[5] "./.config/logs/2020.11.06" "./.ipynb_checkpoints"     
[7] "./sample_data"             "./test"

In [16]:
# 데이터베이스 만들기 위해서 실행
# 2015513_wonyoungAhn으로 데이터베이스파일이 만들어진다. 

con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "test/20155137_wonyoungAhn.sqlite") # 자신의 이름과 학번으로 설정할 것

* local dataframe 생성

In [17]:
# 데이터를 넣어주기 위해서 DF를 생성
# 데이터 만들어주는곳

DF <- tibble(ID = c("20155137"), GRADE = c("3"), NAME = c("wonyougAhn"), RESIDENCE = c("Seoul Dongjak"))
DF %>% show()

# A tibble: 1 x 4
  ID       GRADE NAME       RESIDENCE    
  <chr>    <chr> <chr>      <chr>        
1 20155137 3     wonyougAhn Seoul Dongjak


* copy_to를 이용하여 local dataframe을 SQLite의 테이블로 생성

In [18]:
# 생성한 데이터를 dyplyr을 이용해 데이터베이스에 DF형태로 데이터를 한번에 넣어준다.

dplyr::copy_to(con, DF, "20155137_wonyoungAhn", temporary = F) # connection, 생성한 dataframe, 테이블 명, 일시적인 테이블 해제

* 생성한 테이블 확인

In [19]:
# 테이블이 만들어진것을 확인

dbListTables(con)

[1] "20155137_wonyoungAhn" "sqlite_stat1"         "sqlite_stat4"

In [24]:
# tbl함수를 이용하면 데이터베이스에 있는 데이터를 DF에서 사용하는 것처럼 사용할 수 있다.
tbl(con, "20155137_wonyoungAhn") %>% collect() %>% show()

# A tibble: 1 x 4
  ID       GRADE NAME       RESIDENCE    
  <chr>    <chr> <chr>      <chr>        
1 20155137 3     wonyougAhn Seoul Dongjak


In [25]:
# 데이터 모두다 삭제하지만 구조는 남겨둠

query <- dbSendQuery(con, "DELETE FROM [20155137_wonyoungAhn]")
dbClearResult(query)

In [27]:
# 데이터가 모두 삭제된것을 확인할 수 있다.

dbGetQuery(con, "SELECT * FROM [20155137_wonyoungAhn]")

ID,GRADE,NAME,RESIDENCE
<chr>,<chr>,<chr>,<chr>


In [ ]:
dbDisconnect(con)